In [1]:
# ============================================================================
# PACKAGE INSTALLATION
# ============================================================================
# Install any missing packages
!pip install -q obonet biopython

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 33.3 MB/s eta 0:00:00a 0:00:01


In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
import numpy as np
import pandas as pd
import os

for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# ============================================================================
# CORE IMPORTS
# ============================================================================
import gc
import sys
import subprocess
from pathlib import Path
from collections import defaultdict, Counter
from tqdm.auto import tqdm
import warnings
warnings.filterwarnings('ignore')

# ============================================================================
# CONFIG (ADVANCED IMPROVEMENTS)
# ============================================================================
CONFIG = {
    'MAX_TRAIN_SAMPLES': 90000,        # all data
    'TOP_K_LABELS': 800,
    'RANDOM_SEED': 253,
    'HIDDEN_DIMS': [512, 256, 128],
    'DROPOUT_RATE': 0.27,             # IMPROVED: 0.3 → 0.27 (less dropout for more data)
    'EPOCHS': 36,                     # IMPROVED: 12 - 36 epochs
    'BATCH_SIZE': 64,                 # IMPROVED: 32 → 64 (faster training)
    'LEARNING_RATE': 1e-3,
    'PREDICT_BATCH_SIZE': 128,        # IMPROVED: 64 → 128 (faster prediction)
    'MIN_CONFIDENCE': 0.05,           # IMPROVED: 0.10 → 0.05 (even more predictions)
    'MAX_PREDS_PER_PROTEIN': 200,    # IMPROVED: 150 → 200
    'LABEL_SMOOTHING': 0.08,          # NEW: Add label smoothing
    'TEMPERATURE': 1.4,               # NEW: Temperature scaling for predictions
    'MERGE_WITH_BLAST': True,
    'BLAST_WEIGHT': 0.60,             
    'DL_WEIGHT': 0.40,                
    'BASE_PATH': '/kaggle/input/cafa-6-protein-function-prediction',
    'ESM2_PATH': 'cafa-5-ems-2-embeddings-numpy',
    'BLAST_PATH': '/kaggle/input/blast-quick-sprof-zero-pred/submission.tsv',
}

print("="*80)
print("🚀 CAFA 6 - ADVANCED OPTIMIZATION")
print("="*80)

def install(package):
    subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", package])

try:
    import torch
    import torch.nn as nn
    import torch.nn.functional as F
except:
    install('torch')
    import torch
    import torch.nn as nn
    import torch.nn.functional as F

np.random.seed(CONFIG['RANDOM_SEED'])
torch.manual_seed(CONFIG['RANDOM_SEED'])
if torch.cuda.is_available():
    torch.cuda.manual_seed(CONFIG['RANDOM_SEED'])
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Device: {device}")

# ============================================================================
# LOAD DATA
# ============================================================================
BASE = Path(CONFIG['BASE_PATH'])
TRAIN_DIR = BASE / 'Train'

print("\n[1/5] Loading annotations...")
train_terms_df = pd.read_csv(TRAIN_DIR / 'train_terms.tsv', sep='\t', 
                              header=None, names=['protein', 'term', 'ontology'])
protein_to_terms = train_terms_df.groupby('protein')['term'].apply(list).to_dict()

term_counts = Counter()
for terms in protein_to_terms.values():
    term_counts.update(terms)
top_terms = [t for t, _ in term_counts.most_common(CONFIG['TOP_K_LABELS'])]

for protein in protein_to_terms:
    protein_to_terms[protein] = [t for t in protein_to_terms[protein] if t in top_terms]

term_to_idx = {term: idx for idx, term in enumerate(top_terms)}
print(f"   ✓ {len(protein_to_terms)} proteins, {len(top_terms)} terms")

print("\n[2/5] Loading ESM2 embeddings...")
esm2_base = f"/kaggle/input/{CONFIG['ESM2_PATH']}"
train_ids = np.load(f"{esm2_base}/train_ids.npy", allow_pickle=True)
train_embeds = np.load(f"{esm2_base}/train_embeddings.npy")
test_ids = np.load(f"{esm2_base}/test_ids.npy", allow_pickle=True)
test_embeds = np.load(f"{esm2_base}/test_embeddings.npy")

train_dict = {str(pid): emb for pid, emb in zip(train_ids, train_embeds)}
test_dict = {str(pid): emb for pid, emb in zip(test_ids, test_embeds)}
embed_dim = train_embeds.shape[1]
print(f"   ✓ Dim: {embed_dim}")

del train_ids, train_embeds, test_ids, test_embeds
gc.collect()

print("\n[3/5] Preparing training data...")
valid_proteins = [p for p in protein_to_terms.keys() if p in train_dict][:CONFIG['MAX_TRAIN_SAMPLES']]

from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split

mlb = MultiLabelBinarizer(classes=range(len(top_terms)))
y_labels = [[term_to_idx[t] for t in protein_to_terms.get(p, []) if t in term_to_idx] 
            for p in valid_proteins]
y_encoded = mlb.fit_transform(y_labels)

# Apply label smoothing
if CONFIG['LABEL_SMOOTHING'] > 0:
    y_encoded = y_encoded.astype(float)
    y_encoded = y_encoded * (1 - CONFIG['LABEL_SMOOTHING']) + CONFIG['LABEL_SMOOTHING'] / len(top_terms)

train_proteins, val_proteins, y_train, y_val = train_test_split(
    valid_proteins, y_encoded, test_size=0.15, random_state=CONFIG['RANDOM_SEED']
)
print(f"   ✓ Train: {len(train_proteins)}, Val: {len(val_proteins)}")

# ============================================================================
# MODEL
# ============================================================================
print("\n[4/5] Building model...")

class ImprovedProteinModel(nn.Module):
    """Improved 3-layer architecture with BatchNorm"""
    def __init__(self, input_dim, output_dim, hidden_dims, dropout):
        super().__init__()
        layers = []
        prev_dim = input_dim
        for dim in hidden_dims:
            layers.extend([
                nn.Linear(prev_dim, dim),
                nn.BatchNorm1d(dim),
                nn.ReLU(),
                nn.Dropout(dropout)
            ])
            prev_dim = dim
        self.encoder = nn.Sequential(*layers)
        self.output = nn.Linear(prev_dim, output_dim)
    
    def forward(self, x):
        return torch.sigmoid(self.output(self.encoder(x)))

model = ImprovedProteinModel(embed_dim, len(top_terms), CONFIG['HIDDEN_DIMS'], CONFIG['DROPOUT_RATE']).to(device)
print(f"   ✓ Parameters: {sum(p.numel() for p in model.parameters()):,}")

# ============================================================================
# TRAINING
# ============================================================================
print("\n" + "="*80)
print("TRAINING (12 EPOCHS WITH LABEL SMOOTHING)")
print("="*80)

criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=CONFIG['LEARNING_RATE'], weight_decay=1e-5)  # Added L2 regularization
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=2, verbose=False)

best_val_loss = float('inf')

for epoch in range(CONFIG['EPOCHS']):
    model.train()
    indices = np.random.permutation(len(train_proteins))
    epoch_loss = 0
    n_batches = 0
    
    for i in range(0, len(indices), CONFIG['BATCH_SIZE']):
        batch_idx = indices[i:i + CONFIG['BATCH_SIZE']]
        batch_proteins = [train_proteins[j] for j in batch_idx]
        
        X_batch = torch.FloatTensor([train_dict[p] for p in batch_proteins]).to(device)
        y_batch = torch.FloatTensor(y_train[batch_idx]).to(device)
        
        optimizer.zero_grad()
        outputs = model(X_batch)
        loss = criterion(outputs, y_batch)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)  # Gradient clipping
        optimizer.step()
        
        epoch_loss += loss.item()
        n_batches += 1
    
    model.eval()
    val_loss = 0
    val_batches = 0
    
    with torch.no_grad():
        for i in range(0, len(val_proteins), CONFIG['BATCH_SIZE']):
            batch_proteins = val_proteins[i:i + CONFIG['BATCH_SIZE']]
            X_batch = torch.FloatTensor([train_dict[p] for p in batch_proteins]).to(device)
            y_batch = torch.FloatTensor(y_val[i:i + CONFIG['BATCH_SIZE']]).to(device)
            
            outputs = model(X_batch)
            loss = criterion(outputs, y_batch)
            val_loss += loss.item()
            val_batches += 1
    
    train_loss_avg = epoch_loss/n_batches
    val_loss_avg = val_loss/val_batches
    
    scheduler.step(val_loss_avg)
    
    if val_loss_avg < best_val_loss:
        best_val_loss = val_loss_avg
        print(f"Epoch {epoch+1}: Train={train_loss_avg:.4f}, Val={val_loss_avg:.4f} ⭐ NEW BEST")
    else:
        print(f"Epoch {epoch+1}: Train={train_loss_avg:.4f}, Val={val_loss_avg:.4f}")
    
    gc.collect()

print(f"\n✅ Best Validation Loss: {best_val_loss:.4f}")

# ============================================================================
# PREDICTIONS (WITH TEMPERATURE SCALING)
# ============================================================================
print("\n" + "="*80)
print("PREDICTIONS (WITH TEMPERATURE SCALING)")
print("="*80)

model.eval()
test_protein_ids = list(test_dict.keys())

n_predictions = 0
with open('temp_dl.tsv', 'w', newline='') as f:
    with torch.no_grad():
        for start in tqdm(range(0, len(test_protein_ids), CONFIG['PREDICT_BATCH_SIZE']), desc="Predicting"):
            batch_ids = test_protein_ids[start:start + CONFIG['PREDICT_BATCH_SIZE']]
            X_batch = torch.FloatTensor([test_dict[p] for p in batch_ids]).to(device)
            
            # Get raw logits before sigmoid
            logits = model.encoder(X_batch)
            logits = model.output(logits)
            
            # Apply temperature scaling
            outputs = torch.sigmoid(logits / CONFIG['TEMPERATURE']).cpu().numpy()
            
            for i, pid in enumerate(batch_ids):
                probs = outputs[i]
                top_indices = np.argsort(probs)[::-1][:CONFIG['MAX_PREDS_PER_PROTEIN']]
                confident_indices = [idx for idx in top_indices if probs[idx] > CONFIG['MIN_CONFIDENCE']]
                
                for idx in confident_indices:
                    line = f"{pid}\t{top_terms[idx]}\t{min(probs[idx], 0.999):.3f}\n"
                    f.write(line)
                    n_predictions += 1
            
            del X_batch, outputs, logits
            if start % 1000 == 0:
                gc.collect()

print(f"✓ Generated {n_predictions:,} predictions")

del model, train_dict
gc.collect()

# ============================================================================
# ENSEMBLE
# ============================================================================
print("\n" + "="*80)
print("ENSEMBLE")
print("="*80)

if CONFIG['MERGE_WITH_BLAST'] and os.path.exists(CONFIG['BLAST_PATH']):
    print("Merging with BLAST...")
    
    dl_df = pd.read_csv('temp_dl.tsv', sep='\t', header=None, names=['Id', 'GO term', 'Confidence'])
    dl_df['Confidence'] = dl_df['Confidence'].astype(float) * CONFIG['DL_WEIGHT']
    
    blast_chunks = []
    for chunk in pd.read_csv(CONFIG['BLAST_PATH'], sep='\t', header=None,
                             names=['Id', 'GO term', 'Confidence'], chunksize=1000000):
        chunk['Confidence'] = chunk['Confidence'].astype(float) * CONFIG['BLAST_WEIGHT']
        blast_chunks.append(chunk)
        if len(blast_chunks) >= 10:
            blast_df = pd.concat(blast_chunks, ignore_index=True)
            blast_chunks = [blast_df]
            gc.collect()
    
    blast_df = pd.concat(blast_chunks, ignore_index=True)
    print(f"  DL={len(dl_df):,}, BLAST={len(blast_df):,}")
    
    ensemble_df = pd.concat([dl_df, blast_df], ignore_index=True)
    del dl_df, blast_df, blast_chunks
    gc.collect()
    
    ensemble_df = ensemble_df.groupby(['Id', 'GO term'], as_index=False)['Confidence'].sum()
    ensemble_df = ensemble_df.sort_values(['Id', 'Confidence'], ascending=[True, False])
    ensemble_df = ensemble_df.groupby('Id').head(1500)
    
    import csv
    ensemble_df.to_csv('submission.tsv', sep='\t', header=False, index=False,
                      quoting=csv.QUOTE_NONE, escapechar='\\', lineterminator='\n')
    print(f"✓ Saved {len(ensemble_df):,} predictions")
    
    del ensemble_df
else:
    os.rename('temp_dl.tsv', 'submission.tsv')

gc.collect()

# ============================================================================
# CLEANUP
# ============================================================================
print("\n[CLEANUP]")
for fname in os.listdir('.'):
    if 'submission' in fname.lower() and fname != 'submission.tsv':
        try:
            os.remove(fname)
            print(f"  ✗ Removed {fname}")
        except:
            pass

if os.path.exists('temp_dl.tsv'):
    os.remove('temp_dl.tsv')
    print(f"  ✗ Removed temp_dl.tsv")

# ============================================================================
# VERIFICATION
# ============================================================================
print("\n" + "="*80)
print("✅ VERIFICATION")
print("="*80)

if os.path.exists('submission.tsv'):
    with open('submission.tsv', 'r') as f:
        lines = f.readlines()
    
    n_lines = len(lines)
    file_size_mb = os.path.getsize('submission.tsv') / (1024*1024)
    
    print(f"📋 File: submission.tsv")
    print(f"📊 Lines: {n_lines:,}")
    print(f"💾 Size: {file_size_mb:.1f} MB")
    
    print("\n✅ First 5 lines:")
    for i in range(min(5, len(lines))):
        line = lines[i].strip()
        parts = line.split('\t')
        if len(parts) == 3:
            print(f"  ✓ {parts[0]:<15} {parts[1]:<12} {parts[2]}")
        else:
            print(f"  ✗ Line {i+1}: {len(parts)} columns (expected 3)")
    
    first_line = lines[0].strip()
    parts = first_line.split('\t')
    
    print("\n🔍 Format check:")
    all_good = True
    
    if len(parts) == 3:
        print(f"  ✓ Columns: 3")
    else:
        print(f"  ✗ Columns: {len(parts)} (expected 3)")
        all_good = False
    
    if parts[1].startswith('GO:'):
        print(f"  ✓ GO term format")
    else:
        print(f"  ✗ GO term: {parts[1]}")
        all_good = False
    
    try:
        conf = float(parts[2])
        if 0 < conf <= 1:
            print(f"  ✓ Confidence: {conf}")
        else:
            print(f"  ✗ Confidence out of range: {conf}")
            all_good = False
    except:
        print(f"  ✗ Confidence not a number: {parts[2]}")
        all_good = False
    
    if '\n' in lines[0]:
        print(f"  ✓ Newlines present")
    else:
        print(f"  ✗ No newlines!")
        all_good = False
    
    if all_good:
        print("\n✅✅✅ FORMAT IS CORRECT! ✅✅✅")
    else:
        print("\n⚠️  FORMAT HAS ISSUES!")
        
    print("\n📁 Files in working directory:")
    for fname in sorted(os.listdir('.')):
        if not fname.startswith('.'):
            size = os.path.getsize(fname) / (1024*1024)
            print(f"  {fname} ({size:.1f} MB)")
else:
    print("❌ ERROR: submission.tsv not found!")

print("\n" + "="*80)

/kaggle/input/blast-quick-sprof-zero-pred/submission.tsv
/kaggle/input/protbert-embeddings-for-cafa5/train_ids.npy
/kaggle/input/protbert-embeddings-for-cafa5/train_embeddings.npy
/kaggle/input/protbert-embeddings-for-cafa5/test_ids.npy
/kaggle/input/protbert-embeddings-for-cafa5/test_embeddings.npy
/kaggle/input/train-targets-top500/train_targets_top500.npy
/kaggle/input/cafa-5-ems-2-embeddings-numpy/train_ids.npy
/kaggle/input/cafa-5-ems-2-embeddings-numpy/train_embeddings.npy
/kaggle/input/cafa-5-ems-2-embeddings-numpy/test_ids.npy
/kaggle/input/cafa-5-ems-2-embeddings-numpy/test_embeddings.npy
/kaggle/input/cafa-6-protein-function-prediction/sample_submission.tsv
/kaggle/input/cafa-6-protein-function-prediction/IA.tsv
/kaggle/input/cafa-6-protein-function-prediction/Test/testsuperset.fasta
/kaggle/input/cafa-6-protein-function-prediction/Test/testsuperset-taxon-list.tsv
/kaggle/input/cafa-6-protein-function-prediction/Train/train_terms.tsv
/kaggle/input/cafa-6-protein-function-pred

Predicting:   0%|          | 0/1109 [00:00<?, ?it/s]

✓ Generated 3,363,401 predictions

ENSEMBLE
Merging with BLAST...
  DL=3,363,401, BLAST=11,977,931
✓ Saved 14,090,857 predictions

[CLEANUP]
  ✗ Removed temp_dl.tsv

✅ VERIFICATION
📋 File: submission.tsv
📊 Lines: 14,090,857
💾 Size: 354.4 MB

✅ First 5 lines:
  ✓ A0A009IHW8      GO:0003953   0.6
  ✓ A0A009IHW8      GO:0007165   0.6
  ✓ A0A009IHW8      GO:0016787   0.6
  ✓ A0A009IHW8      GO:0019677   0.6
  ✓ A0A009IHW8      GO:0050135   0.6

🔍 Format check:
  ✓ Columns: 3
  ✓ GO term format
  ✓ Confidence: 0.6
  ✓ Newlines present

✅✅✅ FORMAT IS CORRECT! ✅✅✅

📁 Files in working directory:
  submission.tsv (354.4 MB)

